In [97]:
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import folium, requests, zipfile, io, os
from folium.plugins import MarkerCluster

In [98]:
# download del file urbano
zip_file_url='http://www.ttesercizio.it/opendata/google_transit_urbano_tte.zip'
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
# preparazione dataframe
stops = pd.read_csv('stops.txt')
stop_times = pd.read_csv('stop_times.txt')

In [99]:
# funzione di ricerca
def latlonstopattime(ticketime):
    stopsidattime = stop_times[stop_times.arrival_time == ticketime].stop_id
    stopsattime = stops[stops.stop_id.isin(stopsidattime)]
    return stopsattime

# definizione orario: seguire lo schema HH:MM:00"
ticketime = "12:23:00"

# ricerca fermate
stopattime = latlonstopattime(ticketime)

In [138]:
# conversione in geodataframe
geometry = [Point(xy) for xy in zip(stopattime.stop_lon, stopsattime.stop_lat)]
df = stopattime.drop(['stop_lon', 'stop_lat'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
# calcolo centroide per mappa
center=gdf.unary_union.convex_hull.centroid


In [139]:
# rappresentazione marker cluster con folium
map = folium.Map(location=[center.y,center.x],zoom_start=11,tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(map)
for lat, lon, name in zip(stopsattime.stop_lat, stopsattime.stop_lon, stopsattime.stop_name):
    folium.Marker(
        location=[lat, lon],
        popup=name.replace("'",""),
        icon=folium.Icon(color='green', icon='bus'),
    ).add_to(marker_cluster)
map.fit_bounds(gdf.unary_union.bounds)
map

In [143]:
# salvataggio dei file dal nome stops_orario.geojson o stops_orario.csv
outputgeo = 'stops_'+ ticketime.replace(":","_") + '.geojson'
outputcsv = 'stops_'+ ticketime.replace(":","_") + '.csv'
if os.path.exists(outputgeo):
    os.remove(outputgeo)
if os.path.exists(outputcsv):
    os.remove(outputcsv)
gdf.to_file(outputgeo,driver='GeoJSON')
stopattime.to_csv(outputcsv)